This notebook will walk through the process of fitting $P(\nu,A)$ data to the England and Rider evaluation. The script `fit_P_nu_A.py` can be used to apply this same code to every fissioning system in the `yields` folder.  

First define the fissioning system that will be analyzed (see `yields/systems.txt` for a full list of systems):

In [1]:
system = 'U235F'
Ap = 236
Zp = 92

Import statements:

In [2]:
import numpy
numpy.random.seed(0)
import scipy
from math import erfc
from math import erf
from math import sqrt
from math import exp
from math import pi
from math import isnan
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
from decimal import Decimal
import multiprocessing
import time

The following functions are defined and used in the fitting process. Due to limitations of the `scipy` package they must use some global variables: 

In [3]:
AMIN = 0
AMID = 0
AMAX = 0
ZCN = 0 
ACN = 0
YIELDS = {}

def gauss_trunc(x,mu,sigma):
    """
    Truncated Gaussian function
    x = point along distribution
    mu = centroid of distribution
    sigma = width of distribution 
    """
    #norm = 1.0 / ( 1.0 - ( erfc( mu * sqrt( 1.0 / sigma**2.0 ) / sqrt(2.0) ) / ( 2.0 * mu * sqrt( 1.0 / sigma**2.0 ) ) ) )
    norm = 1.0 / ( 0.5 + 0.5 * erf( mu / (sqrt(2.0)*sigma) ) )
    return norm * ( 1.0 / (sigma * sqrt(2.0*pi)) ) * numpy.exp( -0.5 * ((x-mu)/sigma)**2.0 )

def gauss_trunc_integ(b,c,mu,sigma):
    """
    Definite integral of truncated Gaussian function
    x = point along distribution
    mu = centroid of distribution
    sigma = width of distribution 
    """
    norm = 1.0 / ( 0.5 + 0.5 * erf( mu / (sqrt(2.0)*sigma) ) )
    K = norm * ( 1.0 / (sigma * sqrt(2.0*pi)) )
    return K * sqrt(pi/2.0) * sigma * ( scipy.special.erf( (mu-b)/(sqrt(2.0)*sigma) ) - scipy.special.erf( (mu-c)/(sqrt(2.0)*sigma) ) ) 

def gauss_trunc_int(nu,mu,sigma,mid=0.5):
    """
    Integer value of truncated Gaussian
    nu = integer point of distribution
    mu = centroid of distribution
    sigma = width of distribution 
    mid = midway point to integrate over (nu-(1-mid),nu+mid)
    """
    nu = int(nu)

    lower = nu-(1.0-mid)
    if(lower < 0.0):
        lower = 0.0
    upper = nu+mid

    return gauss_trunc_integ(lower,upper,mu,sigma)

def gauss_trunc_bar(mu,sigma,h=0.001,upper=20):
    """
    Get the expectation value of a truncated Gaussian
    mu = centroid of distribution
    sigma = width of distribution 
    h = integration constant
    upper = max value to integrate up to
    """
    val = 0.0
    for x in numpy.arange(0,upper,h):
        val += gauss_trunc_integ(x,x+h,mu,sigma) * (x+(h/2.0))

    return val

def chi2A(x,return_yields=False):
    """
    Function to calculate chi2 for P_nu_A distribution against ER yields
    Single A chain
    """
    mu = x[0]
    sigma = x[1]
    nu_max = 10
    A_upper = ACN - ACUR
    A_lower = ACN - ACUR - nu_max

    #Calculate yields in A chain
    yields_new = {}
    for key in YIELDS.keys():
        if( (key[1] >= A_lower) and (key[1] <= A_upper) ):
            for j in range(0,nu_max):
                if( (ACN-key[1]-j) == ACUR ):
                    try:
                        yields_new[ZCN-key[0],ACN-key[1]-j] += gauss_trunc_int(j,mu,sigma) * YIELDS[key]
                    except KeyError as e:
                        yields_new[ZCN-key[0],ACN-key[1]-j] = gauss_trunc_int(j,mu,sigma) * YIELDS[key]

    #Calculate chi2
    chi2 = 0.0
    for key in yields_new.keys():
        try:
            chi2 += ( YIELDS[key] - yields_new[key] )**2.0 / YIELDS[key]
        except KeyError as e:
            pass

    if( return_yields ):
        return chi2, yields_new
    else:
        return chi2

def chi2_all_sub(P_data,AMIN_in,AMAX_in,nu_max_in,Zp_in,Ap_in,yields_old_in):
    yields_new = {}
    for key in yields_old_in.keys():
        mu = P_data[ (key[1]-AMIN_in)*2 ]
        sigma = P_data[ (key[1]-AMIN_in)*2 +1 ]
        for j in range(0,nu_max_in):
            try:
                yields_new[Zp_in-key[0],Ap_in-key[1]-j] += gauss_trunc_int(j,mu,sigma) * yields_old_in[key]
            except KeyError as e:
                yields_new[Zp-key[0],Ap-key[1]-j] =  gauss_trunc_int(j,mu,sigma) * yields_old_in[key]

    fom = 0.0
    for A in range(AMIN_in,AMAX_in+1):
        chi2_yields = 0.0
        chi2_P_nu_A = 0.0
        for key in yields_old_in.keys():
            if( key[1] == A ):
                try:
                    chi2_yields += (yields_old_in[key] - yields_new[key])**2.0 / yields_old_in[key]
                except KeyError as e:
                    chi2_yields += yields_old_in[key]
        for j in range(0,nu_max_in):
            mu = P_data[ (A-AMIN_in)*2 ]
            sigma = P_data[ (A-AMIN_in)*2 +1 ]
            A2 = Ap_in - A - j
            mu2 = P_data[ (A2-AMIN_in)*2 ]
            sigma2 = P_data[ (A2-AMIN_in)*2 +1 ]
            if( gauss_trunc_int(j,mu,sigma) > 0.0 ):
                chi2_P_nu_A += ( gauss_trunc_int(j,mu,sigma) - gauss_trunc_int(j,mu2,sigma2) )**2.0 / gauss_trunc_int(j,mu,sigma)
            if( isnan(chi2_P_nu_A) ):
                print('here2')
        fom += chi2_yields * chi2_P_nu_A

    #print(fom)
    return fom
    
def chi2_all(yields_old,Zp,Ap,return_yields=False,guess_in=None):
    """
    Function to calculate chi2 for P_nu_A distribution against ER yields
    """
    nu_max = 10
    AMIN = min( yields_old.keys() )[1]
    AMAX = max( yields_old.keys() )[1]
    if( guess_in == None ):
        guess = [ (0.01,5.0), (0.01,3.0) ] * ((AMAX+1)-AMIN)
    else:
        guess = guess_in[:]
    num_cores = multiprocessing.cpu_count()
    fit_P_nu_A = differential_evolution( chi2_all_sub, guess, args=(AMIN,AMAX,nu_max,Zp,Ap,yields_old), popsize=15*num_cores, updating='deferred', workers=num_cores, strategy='currenttobest1exp' )
    
    P_nu_A = {}
    for A in range(AMIN,AMAX+1):
        P_nu = []
        mu = fit_P_nu_A[(A-AMIN)*2]
        sigma = fit_P_nu_A[(A-AMIN)*2 +1]
        for j in range(0,nu_max):
            P_nu.append( gauss_trunc_int(j,mu,sigma) )
        P_nu_A[A] = P_nu[:]
    
    return P_nu_A

Read in the yields from file: 

In [4]:
file = open( 'yields/' + system + '.csv', 'r' )
lines = file.readlines()
file.close()
A_min = 300
A_max = 0
yields = {}
yields_unc = {}
for line in lines:
    parts = line.split(',')
    Z = int( parts[0] )
    A = int( parts[1] )
    I = int( parts[2] )
    Y = float( parts[3] )
    Y_unc = float( parts[4] )
    if( A < A_min ):
        A_min = A
    if( A > A_max ):
        A_max = A
    if( I == 0 ):
        yields[Z,A] = Y
        yields_unc[Z,A] = Y_unc
YIELDS = yields
YIELDS_UNC = yields_unc
ZCN = Zp
ACN = Ap
AMIN = A_min
AMID = int( (A_max - A_min)/2.0 ) + A_min
AMAX = A_max

In [5]:
fits = []
flags = []
for ACUR in range(AMIN,AMAX+1):
    #Fit values
    #-------------------------------------------------------------------------------------
    guess = [ (0.01,5.0), (0.01,3.0) ]
    num_cores = multiprocessing.cpu_count()
    fit_P_nu_A = differential_evolution( chi2A, guess )
    fit_P_nu_A = fit_P_nu_A.x
    flag = 0
    chi2 = chi2A(fit_P_nu_A)


    #If fit railed on any of the edges, use last A's fit
    if( (fit_P_nu_A[0] == guess[0][0]) or (fit_P_nu_A[0] == guess[0][1]) or (fit_P_nu_A[1] == guess[1][0]) or (fit_P_nu_A[1] == guess[1][1]) or (chi2 == 0.0) ):
        if( fit_last[0] != None ):
            fit_P_nu_A = fit_last
            flag = 1
            chi2 = chi2A(fit_P_nu_A)
    
    fits.append( fit_P_nu_A )
    flags.append( flag )
    fit_last = fit_P_nu_A

guesses = []
dith = 0.25
for i in range(0,len(fits)):
    for j in range(0,2):
        if( flags[i] != 1 ):
            guesses.append( ( fits[i][j]*(1.0-dith), fits[i][j]*(1.0+dith) ) )
        else:
            if( j == 0 ):
                guesses.append( (0.01,5.0) )
            elif( j == 1 ):
                guesses.append( (0.01,3.0) )

In [6]:
time_start = time.time()
P_nu_A = chi2_all(YIELDS,Zp,Ap,guess_in=guesses)
time_end = time.time()
print(time_end-time_start)
print(P_nu_A)

Process ForkPoolWorker-1:
Process ForkPoolWorker-7:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process ForkPoolWorker-8:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cel

  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/usr/local/lib/python3.7/site-packages/scipy/optimize/_differentialevolution.py", line 1012, in __call__
    return self.f(x, *self.args)
  File "<ipython-input-3-9b02edc0a396>", line 103, in chi2_all_sub
    yields_new[Zp_in-key[0],Ap_in-key[1]-j] += gauss_trunc_int(j,mu,sigma) * yields_old_in[key]
  File "/usr/local/lib/python3.7/site-packages/scipy/optimize/_differentialevolution.py", line 1012, in __call__
    return self.f(x, *self.args)
  File "<ipython-input-3-9b02edc0a396>", line 124, in chi2_all_sub
    chi2_P_nu_A += ( gauss_trunc_int(j,mu,sigma) - gauss_trunc_int(j,mu2,sigma2) )**2.0 / gauss_trunc_int(j,mu,sigma)
  File "/usr/local/lib/python3.7/site-packages/scipy/optimize/_differentialevolution.py", line 1012, in __call__
    return self.f(x, *self.args)
  File "<ipython-input-3-9b02edc0a396>", lin

KeyboardInterrupt: 